## Post Call Analytics (PCA) Using Amazon Bedrock

Welcome to this training module on post-call analytics use cases using Amazon Bedrock. 

As businesses continue to interact with customers through various channels, it becomes increasingly important to analyze these interactions to gain insights into customer behavior and preferences. Post-call analytics is one such method that involves analyzing customer interactions after the call has ended. The use of large language models can greatly enhance the effectiveness of post-call analytics by enabling more accurate sentiment analysis, identifying specific customer needs and preferences, and improving overall customer experience. 

In this sample notebook, we will explore following topics to demonstrate the various benefits of using Bedrock for post-call analytics and businesses gain a competitive edge in the modern marketplace.

- Choice of LLM models in Bedrock (Titan Text and Anthropic Claude)
- One model handling multiple PCA tasks
- Handling long call transcripts
- [Stretch] Architecture pattern for production workloads

# Environment Setup
Install and upgrade the packages required to run the sample code. <BR>
**Note: you may need to restart the kernel to use updated packages.**

# Import packages

In [6]:
import langchain
from langchain.llms.bedrock import Bedrock
from langchain import PromptTemplate
import boto3

print(f"langchain version check: {langchain.__version__}")
print(f"boto3 version check: {boto3.__version__}")

langchain version check: 0.0.232
boto3 version check: 1.26.162


# Load transcript files

In [7]:
transcript_files = ["./call_transcripts/negative-refund.txt", 
                    "./call_transcripts/neutral-short.txt",
                    "./call_transcripts/positive-partial-refund.txt"]

transcripts = []

for file_name in transcript_files:
    with open(file_name, "r") as file:
        transcripts.append(file.read())

In [8]:
for i, trans in enumerate(transcripts):
    print(f"transcript #{i+1}: {trans[:300]}\n")
    print("====================\n\n")

transcript #1: timestamp: 2022-12-27 08:26:49.219717

Agent: Thank you for calling our retail support line. My name is ABC. How can I assist you today?

Customer: Yes, I have received a defective product, and I am extremely angry about it! This is unacceptable, and I want it resolved immediately!

Agent: I'm sorry



transcript #2: timestamp: 2023-01-28 08:26:49.219717

Customer: Hi, I'd like to check the balance on my account.

Retail Support: Sure thing! Can I have your account number or phone number associated with the account?

Customer: My phone number is (123) 456-7890.

Retail Support: Great, thank you. Let me pull up y



transcript #3: timestamp: 2022-12-28 08:26:49.219717

Agent: Thank you for calling [Retailer], my name is [Agent Name]. How may I assist you today?

Customer: Hi, I wanted to check on the status of my order. It was supposed to arrive today, but I haven't received it yet.

Agent: I'm sorry to hear that. Can I have 





# Post Call Analysis

## Choice of models in Bedrock
Choose FMs from Amazon, AI21 Labs and Anthropic to find the right FM for your use case.

**Select region: "us-east-1"(M1), "us-west-2"(M2)**

In [9]:
bedrock_region = "us-east-1" 

In [10]:
if bedrock_region == "us-east-1":    
    bedrock_config = {
        "region_name":bedrock_region,
        "endpoint_url":"https://bedrock.us-east-1.amazonaws.com"
    }
elif bedrock_region == "us-west-2":  
    bedrock_config = {
        "region_name":bedrock_region,
        "endpoint_url":"https://prod.us-west-2.frontend.bedrock.aws.dev"
    }

In [30]:
bedrock = boto3.client(
    service_name='bedrock',
    region_name=bedrock_config["region_name"],
    endpoint_url=bedrock_config["endpoint_url"]
)

bedrock_models = {
    "Claude" : "anthropic.claude-v1",
    "TitanText": "amazon.titan-tg1-large", 
    "Claude-instant":"anthropic.claude-instant-v1"
}

max_tokens = {
    "Claude" : 12000,
    "TitanText": 4096,
    "Claude-instant": 9000
}

max_tokens = {"Claude" : 120, "TitanText": 130, "Claude-instant": 120}


In [12]:
from langchain.llms.bedrock import Bedrock
from langchain import PromptTemplate

In [13]:
# Choose one of the bedrock model
model = "TitanText" # "Claude", "TitanText", "Claude-instant"
if model in ["Claude", "Claude-instant"]:
    llm = Bedrock(
        model_id=bedrock_models[model],
        client=bedrock,
        model_kwargs={
            "max_tokens_to_sample":200,
            "stop_sequences":[],
            "temperature":0,
            "top_p":0.9
        },
        endpoint_url='https://prod.us-west-2.frontend.bedrock.aws.dev'
    )
elif model == "TitanText":
    llm = Bedrock(
        model_id=bedrock_models[model],
        client=bedrock,
        model_kwargs={
            "maxTokenCount":4096,
            "stopSequences":[],
            "temperature":0,
            "topP":0.9
        },
        endpoint_url='https://prod.us-west-2.frontend.bedrock.aws.dev'
    )

## Prompt Template
In this notebook, we'll be performing four different analyses(**Summary, Sentiment, Intent and Resolution**), and we'll need a template for each one. 

In [14]:
summary_template = """Analyze the retail support call transcript below. Provide a detail summary of the conversation in complete sentence:

context: "{transcript}"

summary:"""

# What is the sentiment of the conversation: """
sentiment_template = """
This is a sentiment analysis program. What is the customer sentiment using following classes 
["POSITIVE", "NEUTRAL","NEGATIVE"]. classify the conversation into one and exact one of these classes. 
If you don't know or not sure, please use ["NEUTRAL"] class. Do not try to make up a class.

conversation: "{transcript}"

sentiment: """

intent_template = """This is a intent classification program. What is the purpose of the customer call use following 
classes ["SHIPMENT_DELAY", "PRODUCT_DEFECT", "ACCOUNT_QUESTION"]. Classify the conversation into one 
and exact one of these classes. If you don't know, please use ["UNKNOWN"] class. Do not try to make up a class. 

conversation: "{transcript}"

Answer in one word, why is customer calling today: """

resolution_template = """This is a resolution classification program. How did the agent solved the issue use following 
classes ["FULL_REFUND", "PARTIAL_REFUND",  "QUESTION_ANSWERED", "UNRESOLVED"]. classify the conversation into one 
and exact one of these classes. If you don't know, please use ["UNKNOWN"] class. Do not try to make up a class.

conversation: "{transcript}"

Answer in one word, how did the agent resolve the customer question or issue: """

topic_template = """This is topic identification program. What specific topic agent observed during the call. If you don't know, please say "I don't know". Do not try to make up.

conversation: "{transcript}"

Topic: """

escalation_template = """This is escalation classification program. Did customer asked for escalation during the call. use following classes ["YES", "NO",  "UNKNOWN"]. Classify the conversation into one 
and exact one of these classes. Answer in one word without any explaination. If you don't know, please use ["UNKNOWN"] class. Do not try to make up a class.

conversation: "{transcript}"

Escalation: """


holdup_template = """This is delay identification program. Did agent put customer on hold during the call. 
If Yes, provide top reason concisely. If no wait or hold, then just say "No Hold-up".

conversation: "{transcript}"

Top Reason: 

"""

## Generate Analysis

In [15]:
def generate_analysis(llm, transcript, max_tokens=50, template=""):

    prompt = PromptTemplate(template=template, input_variables=["transcript"])
    
    analysis_prompt = prompt.format(transcript=transcript)
    print (analysis_prompt)
        
    analysis = llm(analysis_prompt)
    
    return analysis

### Summary

In [16]:
generate_analysis(llm=llm, transcript=transcripts[1], template=summary_template)

Analyze the retail support call transcript below. Provide a detail summary of the conversation in complete sentence:

context: "timestamp: 2023-01-28 08:26:49.219717

Customer: Hi, I'd like to check the balance on my account.

Retail Support: Sure thing! Can I have your account number or phone number associated with the account?

Customer: My phone number is (123) 456-7890.

Retail Support: Great, thank you. Let me pull up your account. Okay, it looks like your current balance is $567.89.

Customer: Okay, great. Thank you.

Retail Support: You're welcome! Is there anything else I can help you with today?

Customer: No, that's all. Thank you.

Retail Support: Not a problem. Thank you for calling. Have a great day!

Customer: You too. Goodbye.

Retail Support: Goodbye!"

summary:


'\nThe customer called to check their account balance and the retail support representative provided the current balance of $567.89. The customer was satisfied with the response and there was no further discussion or assistance required. The call ended shortly after with a thank you from both parties.'

### Sentiment Analysis

In [17]:
generate_analysis(llm=llm, transcript=transcripts[1], template=sentiment_template)


This is a sentiment analysis program. What is the customer sentiment using following classes 
["POSITIVE", "NEUTRAL","NEGATIVE"]. classify the conversation into one and exact one of these classes. 
If you don't know or not sure, please use ["NEUTRAL"] class. Do not try to make up a class.

conversation: "timestamp: 2023-01-28 08:26:49.219717

Customer: Hi, I'd like to check the balance on my account.

Retail Support: Sure thing! Can I have your account number or phone number associated with the account?

Customer: My phone number is (123) 456-7890.

Retail Support: Great, thank you. Let me pull up your account. Okay, it looks like your current balance is $567.89.

Customer: Okay, great. Thank you.

Retail Support: You're welcome! Is there anything else I can help you with today?

Customer: No, that's all. Thank you.

Retail Support: Not a problem. Thank you for calling. Have a great day!

Customer: You too. Goodbye.

Retail Support: Goodbye!"

sentiment: 


'NEUTRAL'

### Intent Analysis

In [18]:
generate_analysis(llm=llm, transcript=transcripts[1], template=intent_template)

This is a intent classification program. What is the purpose of the customer call use following 
classes ["SHIPMENT_DELAY", "PRODUCT_DEFECT", "ACCOUNT_QUESTION"]. Classify the conversation into one 
and exact one of these classes. If you don't know, please use ["UNKNOWN"] class. Do not try to make up a class. 

conversation: "timestamp: 2023-01-28 08:26:49.219717

Customer: Hi, I'd like to check the balance on my account.

Retail Support: Sure thing! Can I have your account number or phone number associated with the account?

Customer: My phone number is (123) 456-7890.

Retail Support: Great, thank you. Let me pull up your account. Okay, it looks like your current balance is $567.89.

Customer: Okay, great. Thank you.

Retail Support: You're welcome! Is there anything else I can help you with today?

Customer: No, that's all. Thank you.

Retail Support: Not a problem. Thank you for calling. Have a great day!

Customer: You too. Goodbye.

Retail Support: Goodbye!"

Answer in one word, 

'ACCOUNT_QUESTION'

### Resolution Analysis

In [19]:
generate_analysis(llm=llm, transcript=transcripts[1], template=resolution_template)

This is a resolution classification program. How did the agent solved the issue use following 
classes ["FULL_REFUND", "PARTIAL_REFUND",  "QUESTION_ANSWERED", "UNRESOLVED"]. classify the conversation into one 
and exact one of these classes. If you don't know, please use ["UNKNOWN"] class. Do not try to make up a class.

conversation: "timestamp: 2023-01-28 08:26:49.219717

Customer: Hi, I'd like to check the balance on my account.

Retail Support: Sure thing! Can I have your account number or phone number associated with the account?

Customer: My phone number is (123) 456-7890.

Retail Support: Great, thank you. Let me pull up your account. Okay, it looks like your current balance is $567.89.

Customer: Okay, great. Thank you.

Retail Support: You're welcome! Is there anything else I can help you with today?

Customer: No, that's all. Thank you.

Retail Support: Not a problem. Thank you for calling. Have a great day!

Customer: You too. Goodbye.

Retail Support: Goodbye!"

Answer in 

'QUESTION_ANSWERED'

In [20]:
generate_analysis(llm=llm, transcript=transcripts[1], template=topic_template)

This is topic identification program. What specific topic agent observed during the call. If you don't know, please say "I don't know". Do not try to make up.

conversation: "timestamp: 2023-01-28 08:26:49.219717

Customer: Hi, I'd like to check the balance on my account.

Retail Support: Sure thing! Can I have your account number or phone number associated with the account?

Customer: My phone number is (123) 456-7890.

Retail Support: Great, thank you. Let me pull up your account. Okay, it looks like your current balance is $567.89.

Customer: Okay, great. Thank you.

Retail Support: You're welcome! Is there anything else I can help you with today?

Customer: No, that's all. Thank you.

Retail Support: Not a problem. Thank you for calling. Have a great day!

Customer: You too. Goodbye.

Retail Support: Goodbye!"

Topic: 


' Checking account balance'

In [21]:
generate_analysis(llm=llm, transcript=transcripts[1], template=escalation_template)

This is escalation classification program. Did customer asked for escalation during the call. use following classes ["YES", "NO",  "UNKNOWN"]. Classify the conversation into one 
and exact one of these classes. Answer in one word without any explaination. If you don't know, please use ["UNKNOWN"] class. Do not try to make up a class.

conversation: "timestamp: 2023-01-28 08:26:49.219717

Customer: Hi, I'd like to check the balance on my account.

Retail Support: Sure thing! Can I have your account number or phone number associated with the account?

Customer: My phone number is (123) 456-7890.

Retail Support: Great, thank you. Let me pull up your account. Okay, it looks like your current balance is $567.89.

Customer: Okay, great. Thank you.

Retail Support: You're welcome! Is there anything else I can help you with today?

Customer: No, that's all. Thank you.

Retail Support: Not a problem. Thank you for calling. Have a great day!

Customer: You too. Goodbye.

Retail Support: Goodbye

'NO'

In [22]:
generate_analysis(llm=llm, transcript=transcripts[1], template=holdup_template)

This is delay identification program. Did agent put customer on hold during the call. 
If Yes, provide top reason concisely. If no wait or hold, then just say "No Hold-up".

conversation: "timestamp: 2023-01-28 08:26:49.219717

Customer: Hi, I'd like to check the balance on my account.

Retail Support: Sure thing! Can I have your account number or phone number associated with the account?

Customer: My phone number is (123) 456-7890.

Retail Support: Great, thank you. Let me pull up your account. Okay, it looks like your current balance is $567.89.

Customer: Okay, great. Thank you.

Retail Support: You're welcome! Is there anything else I can help you with today?

Customer: No, that's all. Thank you.

Retail Support: Not a problem. Thank you for calling. Have a great day!

Customer: You too. Goodbye.

Retail Support: Goodbye!"

Top Reason: 




'No Hold-up'

## Handling long call transcripts
We'll cover how to handle long transcripts that exceed the limits of the LLM. 

In [23]:
# Check if exceeds titan limit of 4000 tokens; Chunk it up
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [27]:
# Generate Analysis
def generate_analysis(llm, transcript, template="", model="Claude"):

    prompt = PromptTemplate(template=template, input_variables=["transcript"])
    
    analysis_prompt = prompt.format(transcript=transcript)
    
    num_tokens = llm.get_num_tokens(analysis_prompt)
    print (f'prompt has almost {num_tokens} tokens \n')
    
    print ("max_tokens[model]", max_tokens[model])
    if num_tokens > max_tokens[model]:
        text_splitter = RecursiveCharacterTextSplitter(
            separators=["\n\n", "\n"],
            chunk_size=500,
            chunk_overlap=20
        )
        docs = text_splitter.create_documents([transcript])
        
        num_docs = len(docs)
        num_tokens_first_doc = llm.get_num_tokens(docs[0].page_content)

        print(
            f"Now we have {num_docs} documents and the first one has {num_tokens_first_doc} tokens"
        )
        summary_chain = load_summarize_chain(llm=llm, chain_type="refine", verbose=True) # map_reduce
        
        transcript = summary_chain.run(docs)
                
    analysis_prompt = prompt.format(transcript=transcript) 
    analysis = llm(analysis_prompt)
    
    return (analysis)

### Summary

In [31]:
generate_analysis(llm=llm, transcript=transcripts[1], template=summary_template)

prompt has almost 219 tokens 

max_tokens[model] 120
Now we have 2 documents and the first one has 140 tokens


> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"timestamp: 2023-01-28 08:26:49.219717

Customer: Hi, I'd like to check the balance on my account.

Retail Support: Sure thing! Can I have your account number or phone number associated with the account?

Customer: My phone number is (123) 456-7890.

Retail Support: Great, thank you. Let me pull up your account. Okay, it looks like your current balance is $567.89.

Customer: Okay, great. Thank you.

Retail Support: You're welcome! Is there anything else I can help you with today?"


CONCISE SUMMARY:

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Your job is to produce a final summary
We have provided an existing summary up to a certain point: 
The customer calls to check their account balance and t

' The customer inquired about their account balance, and the retail support representative provided the information.'

In [26]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 110.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 146.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 107.6 MB/s eta 0:00:00


### Sentiment Analysis

In [58]:
generate_analysis(llm=llm, transcript=transcripts[1], template=sentiment_template)


This is a sentiment analysis program. What is the customer sentiment using following classes 
["POSITIVE", "NEUTRAL","NEGATIVE"]. classify the conversation into one and exact one of these classes. 
If you don't know or not sure, please use ["NEUTRAL"] class. Do not try to make up a class.

conversation: "timestamp: 2023-01-28 08:26:49.219717

Customer: Hi, I'd like to check the balance on my account.

Retail Support: Sure thing! Can I have your account number or phone number associated with the account?

Customer: My phone number is (123) 456-7890.

Retail Support: Great, thank you. Let me pull up your account. Okay, it looks like your current balance is $567.89.

Customer: Okay, great. Thank you.

Retail Support: You're welcome! Is there anything else I can help you with today?

Customer: No, that's all. Thank you.

Retail Support: Not a problem. Thank you for calling. Have a great day!

Customer: You too. Goodbye.

Retail Support: Goodbye!"

sentiment: 


ValueError: Error raised by bedrock service: An error occurred (ThrottlingException) when calling the InvokeModel operation (reached max retries: 4): Too many requests, please wait before trying again. You have sent too many requests.  Wait before trying again.

### Intent Analysis

In [ ]:
generate_analysis(llm=llm,transcript=transcripts[1], template=intent_template)

### Resolution Analysis

In [ ]:
generate_analysis(llm=llm, transcript=transcripts[1], template=resolution_template)